In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.7MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from ast import literal_eval
from tqdm import tqdm_notebook

import eli5
from eli5.sklearn import PermutationImportance

In [3]:
cd /content/drive/My Drive/Colab Notebooks/dataworkshop_matrix/shoes/data

/content/drive/My Drive/Colab Notebooks/dataworkshop_matrix/shoes/data


In [5]:
df = pd.read_csv('shoesprices_usd.csv', low_memory=False)
df.shape

(18280, 48)

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower() ).factorize()[0]

In [0]:
feats = df.select_dtypes(include=[np.number]).columns

def run_model(feats, model = DecisionTreeRegressor(max_depth= 5) ):
  X = df[ feats ].values
  y = df[ 'prices_amountmin' ].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error' )
  return np.mean(scores), np.std(scores)

In [20]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state = 0)
run_model( ['brand_cat'], model )

(-57.31783843165656, 4.181246596160967)

In [46]:
#Feature Engineering
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  #slownik na wejsciu
  output_dict = {}
  #jesli wartosc pusta
  if str(x) == 'nan': return output_dict
  features =  literal_eval(x.replace('\\"', '"'))
  for item in features:
    #wszystko z malej litery plus usuwamy spacje
    key =item['key'].lower().strip()
    value=item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict



df['features_parsed'] = df['features'].map(parse_features)

In [73]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )

len(keys)

476

In [77]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [78]:
#There are different keys in the dictionary so we have to create a function 
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats:feats[key] if key in feats else np.nan)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [86]:
{k:v for k,v in keys_stat.items() if v > 50 }

{'gender': 50.17505470459519}

In [87]:
{k:v for k,v in keys_stat.items() if v > 30 }

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
#Factorizng all values below 30%
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat__manufacturer part numbercat_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

In [89]:
df[df.brand != df.feat_brand].shape

(18228, 531)

In [90]:
df[df.brand == df.feat_brand].shape

(52, 531)

In [94]:
df[df.brand == df.feat_brand ][ ['brand', 'feat_brand'] ].head()

,brand,feat_brand
2163,adidas,adidas
2417,adidas,adidas
3021,adidas,adidas
3054,adidas,adidas
3629,totes,totes


In [95]:
df[df.brand != df.feat_brand ][ ['brand', 'feat_brand'] ].head()

,brand,feat_brand
0,Josmo,josmo
1,Josmo,josmo
2,SERVUS BY HONEYWELL,servus by honeywell
3,SERVUS BY HONEYWELL,servus by honeywell
4,SERVUS BY HONEYWELL,servus by honeywell


In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

In [97]:
df[df.brand == df.feat_brand].shape

(8846, 531)

In [98]:
#Random Forest
feats = ['']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.32539782838021, 4.209100023698827)

In [99]:
feats = ['brand_cat', 'feat_brand_cat','feat_color_cat','feat_gender_cat','feat__manufacturer part numbercat_cat','feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.100906100004394, 4.257262347433273)

In [100]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
m.fit(X,y)

perm=PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2565 ± 0.0110,brand_cat
0.1057 ± 0.0095,feat_material_cat
0.0431 ± 0.0043,feat_gender_cat
0.0204 ± 0.0010,feat_brand_cat
0.0072 ± 0.0004,feat_color_cat
0.0008 ± 0.0001,feat__manufacturer part numbercat_cat


In [101]:
#We'd like to delete the weakest feature
feats = ['brand_cat', 'feat_brand_cat','feat_color_cat','feat_gender_cat','feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.067919976353984, 4.141310800281186)

In [102]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
m.fit(X,y)

perm=PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2581 ± 0.0103,brand_cat
0.1015 ± 0.0115,feat_material_cat
0.0438 ± 0.0020,feat_gender_cat
0.0204 ± 0.0003,feat_brand_cat
0.0065 ± 0.0013,feat_color_cat


In [103]:
df['brand'].value_counts()

nike            1777
puma             609
ralph lauren     526
vans             386
new balance      371
                ... 
lyceem             1
merona             1
sport wrap         1
dim                1
jack spade         1
Name: brand, Length: 1732, dtype: int64

In [104]:
df['brand'].value_counts(normalize=True)

nike            0.097210
puma            0.033315
ralph lauren    0.028775
vans            0.021116
new balance     0.020295
                  ...   
lyceem          0.000055
merona          0.000055
sport wrap      0.000055
dim             0.000055
jack spade      0.000055
Name: brand, Length: 1732, dtype: float64

In [105]:
df[ df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [0]:
df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

In [107]:
feats = ['brand_cat', 'feat_brand_cat','feat_gender_cat','feat_material_cat','feat_style_cat','feat_sport_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.05844648536811, 4.242900481832938)

In [108]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
m.fit(X,y)

perm=PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2600 ± 0.0110,brand_cat
0.1047 ± 0.0084,feat_material_cat
0.0458 ± 0.0047,feat_gender_cat
0.0213 ± 0.0011,feat_brand_cat
0.0069 ± 0.0016,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [109]:
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{},
       {'style': 'basketball shoes', 'authenticity': '100% authentic guaranteed', 'material': 'synthetic', 'country/region of manufacture': 'china', 'condition': 'new with box'},
       {'sport': 'soccer', 'shade': 'white-hyper turquoise', 'condition': 'new with box'},
       {},
       {'sport': 'basketball', 'style': 'basketball shorts', 'condition': 'new with tags'}],
      dtype=object)

In [110]:
df[ 'feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
mens                   4
toddler                4
infant                 4
boys'                  3
women ,�� unisex       2
youth                  2
men||women             2
women                  2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64

In [0]:
for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [112]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_recommended location',
 'feat_certifications and listings',
 'feat_clothing category',
 'feat_shoe category',
 'feat_location - city/state',
 'feat_location - country',
 'feat_catalog',
 'feat_multi pack indicator',
 'feat_fabrication',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat__manufacturer part numbercat_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_outer material_cat',
 'feat_diameter_cat',
 'feat_place of origin_cat',
 'feat_pant style_cat',
 'feat_diamond color_cat',
 'feat_chest size_cat',
 'feat_waist size_cat',
 'feat_country/region of manufacture_cat',
 'feat_light weight_cat',
 'feat_character_cat',
 'feat_handbag style_cat',
 'feat_clothing product type_cat',
 'feat_age gender group_cat',
 'feat_accessory type_cat',
 'feat_shirt size_cat',
 'feat_condition_cat',
 'feat_eye protection type_cat',
 'feat_amazonbestsellersrank_cat',
 'feat_quantity in set_cat',
 'feat_animal type_cat',
 'feat_f

In [0]:
feats = ['brand_cat','feat_brand_cat','feat_weight_cat','feat_gender_cat','feat_material_cat','feat_style_cat','feat_sport_cat']

In [0]:

feats += ['feat_manufacturer part number_cat',
 'feat_����������������_cat',
 'feat_number of items_cat',
 'feat_sock size_cat',
 'feat_alarm_cat',
 'feat_sleeve style_cat',
 'feat_country//region of manufacture_cat',
 'feat_waist_cat',
 'feat_assembled product weight_cat',
 'feat_is wheeled_cat',
 'feat_watch case shape_cat',
 'feat_band width_cat',
 'feat_hooded_cat',
 'feat_closure style_cat',
 'feat_lens color_cat',
 'feat_resolution megapixels_cat',
 'feat_upper material_cat',
 'feat_isbn_cat',
 'feat_style #_cat',
 'feat_band length_cat',
 'feat_series name_cat',
 'feat_autographed_cat',
 'feat_fabric care instructions_cat',
 'feat_is portable_cat',
 'feat_designed for_cat',
 'feat_100% adidas authentic_cat',
 'feat_is dark sky-compliant_cat',
 'feat_measurements:_cat',
 'feat_colour_cat',
 'feat_foot arch_cat',
 'feat_ring style_cat',
 'feat_item spec attr - style (ebay)_cat',
 'feat_lining material_cat',
 'feat_country of origin assembly:_cat',
 'feat_general warranty_cat',
 'feat_leg opening:_cat',
 'feat_sku_cat',
 'feat_interior pockets_cat',
 'feat_age gender group_cat',
 'feat_international shipping_cat',
 'feat_uv rating_cat',
 'feat_airport friendly_cat',
 'feat_polarised_cat',
 'feat_primary color_cat',
 'feat_mirrored_cat',
 'feat_insulation_cat',
 'feat_has adaptive lenses_cat',
 'feat_number of heat settings_cat',
 'feat_finish_cat',
 'feat_has paper wood_cat',
 'feat_impact resistant_cat',
 'feat_clothing size type_cat',
 'feat_clothing type_cat',
 'feat_batteries included?_cat',
 'feat_bridge size:_cat',
 'feat_used with equipment type_cat',
 'feat_article_cat',
 'feat_manufacturer_part_number_cat',
 'feat_nike id_cat',
 'feat_age group_cat',
 'feat_eye size:_cat',
 'feat_chain length_cat',
 'feat_gemstone type_cat',
 'feat_fabric material_cat',
 'feat_shoe fastening type_cat',
 'feat_leather grade_cat',
 'feat_fabric_cat',
 'feat_era_cat',
 'feat_stone type_cat',
 'feat_enhanced visibility (reflective)_cat',
 'feat_lined_cat',
 'feat_size_cat',
 'feat_dial material_cat',
 'feat_polarized lenses_cat',
 'feat_fit:_cat',
 'feat_pocket_cat',
 'feat_footwear type_cat',
 'feat_has mercury_cat',
 'feat_use_cat',
 'feat_band manufacturer_cat',
 'feat_features & fastening_cat',
 'feat_design_cat',
 'feat_frame type_cat',
 'feat_number of pieces_cat',
 'feat_jacket length_cat',
 'feat_wheel type_cat',
 'feat_bag size range_cat',
 'feat_item type_cat',
 'feat_width_cat',
 'feat_case back_cat',
 'feat_weight_cat',
 'feat_feature_cat',
 'feat_primary shelf id_cat',
 'feat_clothing category_cat',
 'feat_inv_cat',
 'feat_item weight unit of measure_cat',
 'feat_arm_cat',
 'feat_multi pack indicator_cat',
 'feat_lens width_cat',
 'feat_fabric content_cat',
 'feat_country of manufacture_cat',
 'feat_work shoes_cat',
 'feat_features_cat',
 'feat_weather-resistant_cat',
 'feat_hood_cat',
 'feat_stock no._cat',
 'feat_watch band material_cat',
 'feat_purpose_cat',
 'feat_shade_cat',
 'feat_overall material_cat',
 'feat_carats_cat',
 'feat_hat style_cat',
 'feat_golf shoe type_cat',
 'feat_date first available at amazon.ca_cat',
 'feat_material detail-1_cat',
 'feat_compatible devices_cat',
 'feat_sizearm_cat',
 'feat_case thickness_cat',
 'feat_hairstyle_cat',
 'feat_accessory type_cat',
 'feat_movement_cat',
 'feat_fastener type_cat',
 'feat_has expiration_cat',
 'feat_animal type_cat',
 'feat_release_cat',
 'feat_part number_cat',
 'feat_dial color_cat',
 'feat_chain/necklace length (in.)_cat',
 'feat_shoe closure_cat',
 'feat_gold karat_cat',
 'feat_diamond color_cat',
 'feat_chronograph_cat',
 'feat_wash_cat',
 'feat_year_cat',
 'feat_ul safety listing_cat',
 'feat_sub style_cat',
 'feat_front_cat',
 'feat_protects against_cat',
 'feat_front style_cat',
 'feat_wind resistant_cat',
 'feat_материал_cat',
 'feat_fishing_cat',
 'feat_water-resistant_cat',
 'feat_suitable for_cat',
 'feat_hammer loop_cat',
 'feat_product in inches (l x w x h)_cat',
 'feat_spikes type_cat',
 'feat_fits model_cat',
 'feat_number of pockets_cat',
 'feat_packageweight_cat',
 'feat_age start_cat',
 'feat_made in_cat',
 'feat_product id_cat',
 'feat_reinforced knee_cat',
 'feat_bridge/temple size:_cat',
 'feat_safety features_cat',
 'feat_batteries required_cat',
 'feat_rise_cat',
 'feat_origin of components_cat',
 'feat_chain included_cat',
 'feat_face shape_cat',
 'feat_heart rate monitor_cat',
 'feat_shipping to usa_cat',
 'feat_shoe width_cat',
 'feat_lining_cat',
 'feat_date first available_cat']

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [118]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
m.fit(X,y)

print(result)
perm=PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

(-57.4672703504129, 4.392551469350359)


Weight,Feature
0.2534 ± 0.0031,brand_cat
0.1019 ± 0.0152,feat_material_cat
0.0143 ± 0.0015,feat_brand_cat
0.0128 ± 0.0018,feat_gender_cat
0.0119 ± 0.0007,feat_fabric content_cat
0.0054 ± 0.0006,feat_weight_cat
0.0048 ± 0.0003,feat_fabric material_cat
0.0041 ± 0.0007,feat_case thickness_cat
0.0041 ± 0.0003,feat_weight_cat
0.0038 ± 0.0005,feat_gold karat_cat
